In [ ]:
from datetime import datetime
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from time import sleep as wait

PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

## Parameters for the search
words2search = ["Algorand", "ido"]

## Search URL from the parameters
url = f'https://www.reddit.com/search/?q={words2search[0]}%20and%20{words2search[1]}'
driver.get(url)

wait(20)
print("Page ready !")

#Set filter parameters on all the posts that appreared:
stringys = ["hiring", "ido", "funding", "upcoming"]

#Create tweet ID's for already scraped tweets
post_ids = set()
post_data = []

#Check if I'm at the end of the page
last_pos = driver.execute_script("return document.body.scrollHeight")

#Set scrolilng
scrolling = True
in_range = True

#Looping through posts
while scrolling:
    ## Get the post's card
    cards = driver.find_elements(By.CLASS_NAME, "Post")
    
    for card in cards:
        
        try: 
            text = card.find_element_by_xpath('./div/div/div[2]/div[1]/div/div[1]/a/div/h3').text
            link = card.find_element_by_xpath(".//a[@data-click-id='body']").get_attribute('href')
            time = card.find_element_by_xpath(".//span[@data-testid='post_timestamp']").text
            #timestamp = int(time.split(" ")[1])
            #print(timestamp)
            
            """STR date into python datetime object"""
            
        #NoSuchElement error handling
        except NoSuchElementException:
            pass
        
        #StaleElementReferenceException error handling
        except StaleElementReferenceException:
            pass
        
        if any(s in text.lower() for s in stringys):

            if "year" not in time:
                #Create a tuple for the post
                post = (text, link, time)

                #Create the tweet DF
                post_id = ''.join(post)

                #Add only tweets not already seen
                if post_id not in post_ids:
                    #Add id & data
                    post_ids.add(post_id)
                    post_data.append(post)
                    
        if "year" in time:
            scrolling = False
            in_range = False
            print("Post out of range")
            break
            
    scrolling_attempt = 0
    
    while in_range == True:
        #Finally adding pagination
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        wait(5)

        #Current position and comparison to check if I'm at the bottom
        curr_pos = driver.execute_script("return document.body.scrollHeight")
        
        if last_pos == curr_pos: 
            #If I scrolled and ended up in the same place //nothing loaded//
            scrolling_attempt +=1
            
            #End of scroll region
            if scrolling_attempt >= 3: 
                scrolling = False
                break
            else:
                print(f'{scrolling_attempt}^st Attempt to scroll. Break in 3rd.')
                wait(10)
        else:
            last_pos = curr_pos
            break
    else:
        break
        
"""Results"""
print("---------------------------------------------------")
print(f'Amount of posts collected: {len(post_data)}')
print("---------------------------------------------------")
for i in post_data: 
    print("---------0---------")
    print(i)

driver.quit()
        